In [1]:
import requests
from bs4 import BeautifulSoup
from yahoofinancials import YahooFinancials
import yfinance
import pandas as pd
import datetime 
import concurrent.futures as cf
import multiprocessing

In [2]:
url = "https://www.anfin.vn/blog/danh-sach-ma-co-phieu-theo-nganh"
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html5lib')

In [3]:
tickers = [] 
for table in soup.find_all(name = "table"):
    for row in table.find_all(name = "p"):
        if len(row.text) == 3:
           tickers.append(row.text)
print(len(tickers)) 

641


In [4]:
stocks = set([stock + ".VN" for stock in tickers])
len(stocks)

602

In [5]:
balanceSheet = {}
incomeStatement = {}
cashStatement = {}

for stock in stocks:
    print(stock)
    financials_info = YahooFinancials(stock)
    balance_sheet_data = financials_info.get_financial_stmts("annual", "balance")
    income_statement_data = financials_info.get_financial_stmts("annual", "income")
    cash_statement_data = financials_info.get_financial_stmts("annual", "cash")
    
    balanceSheet[stock] = balance_sheet_data["balanceSheetHistory"][stock]
    incomeStatement[stock] = income_statement_data["incomeStatementHistory"][stock]
    cashStatement[stock] = cash_statement_data["cashflowStatementHistory"][stock]

    break

HSV.VN


In [6]:
balanceSheet

{'HSV.VN': []}

In [7]:
balanceSheet = {}
incomeStatement = {}
cashStatement = {}

def retrive_stock_data(stock, balanceSheet = balanceSheet, incomeStatement = incomeStatement, cashStatement = cashStatement):
    try:
        financials_info = YahooFinancials(stock)
        balance_sheet_data = financials_info.get_financial_stmts("annual", "balance")
        income_statement_data = financials_info.get_financial_stmts("annual", "income")
        cash_statement_data = financials_info.get_financial_stmts("annual", "cash")
        
        balanceSheet[stock] = balance_sheet_data["balanceSheetHistory"][stock]
        incomeStatement[stock] = income_statement_data["incomeStatementHistory"][stock]
        cashStatement[stock] = cash_statement_data["cashflowStatementHistory"][stock]
        print(f"Succesfully retrived {stock}'s data")
    except:
        print(f"Error in retriving {stock}'s data")

In [8]:
executor = cf.ThreadPoolExecutor(16)
futures = [executor.submit(retrive_stock_data, stock) for stock in stocks]
cf.wait(futures)

Succesfully retrived NDP.VN's data
Succesfully retrived VMI.VN's data
Succesfully retrived NJC.VN's data
Succesfully retrived KSE.VN's data
Succesfully retrived HHG.VN's data
Succesfully retrived PCG.VN's data
Succesfully retrived AMV.VN's data
Succesfully retrived SZC.VN's data
Succesfully retrived NTL.VN's data
Succesfully retrived REE.VN's data
Succesfully retrived PEQ.VN's data
Succesfully retrived HSV.VN's data
Succesfully retrived LGM.VN's data
Succesfully retrived TNC.VN's data
Succesfully retrived AFX.VN's data
Succesfully retrived NT2.VN's data
Succesfully retrived VHG.VN's data
Succesfully retrived KMT.VN's data
Succesfully retrived OCH.VN's data
Succesfully retrived WSS.VN's data
Succesfully retrived POW.VN's data
Succesfully retrived IJC.VN's data
Succesfully retrived VRG.VN's data
Succesfully retrived BVH.VN's data
Succesfully retrived NBB.VN's data
Succesfully retrived PPP.VN's data
Succesfully retrived BRS.VN's data
Succesfully retrived DL1.VN's data
Succesfully retrived

DoneAndNotDoneFutures(done={<Future at 0x7fbf3c3a8050 state=finished returned NoneType>, <Future at 0x7fbf381e0090 state=finished returned NoneType>, <Future at 0x7fbf3c3b00d0 state=finished returned NoneType>, <Future at 0x7fbf381f00d0 state=finished returned NoneType>, <Future at 0x7fbf3c3d8190 state=finished returned NoneType>, <Future at 0x7fbf3c3501d0 state=finished returned NoneType>, <Future at 0x7fbf3c3e01d0 state=finished returned NoneType>, <Future at 0x7fbf381c8210 state=finished returned NoneType>, <Future at 0x7fbf3c3a0250 state=finished returned NoneType>, <Future at 0x7fbf3c3a8290 state=finished returned NoneType>, <Future at 0x7fbf381e0290 state=finished returned NoneType>, <Future at 0x7fbf3c3b02d0 state=finished returned NoneType>, <Future at 0x7fbf381f02d0 state=finished returned NoneType>, <Future at 0x7fbf3c350350 state=finished returned NoneType>, <Future at 0x7fbf3c3d8390 state=finished returned NoneType>, <Future at 0x7fbf3c3e03d0 state=finished returned NoneTyp

In [10]:
count = 0
for k, v in balanceSheet.items():
    if v != []:
        count += 1
print(count)

83


In [ ]:
roe_dict, epsg_dict = {}, {}
count_missing, count_cond, count_eps_0 = 0, 0, 0
for (keyB, valB), (keyI, valI) in zip(balanceSheet.items(), incomeStatement.items()):
    
    try:
        if keyB == keyI:
            yearI = [k for year in valI for k, v in year.items()]
            yearB = [k for year in valB for k, v in year.items()]
            
            if yearI == yearB:
                count_cond += 1
                equity = [v["totalStockholderEquity"] for year in valB for k, v in year.items()]
                commonStock = [v["commonStock"] for year in valB for k, v in year.items()]
                
                profit =  [v["grossProfit"] for year in valI for k, v in year.items()]
                revenue = [v["totalRevenue"] for year in valI for k, v in year.items()]
                netIncome = [v["netIncome"] for year in valI for k, v in year.items()]
                
                roe = [round(netin/equity * 100, 2) for netin, equity in zip(netIncome, equity)]
                roe_dict[keyB] = round(sum(roe) / len(roe), 2)
                
                eps = [round(earn/common_stock, 2) for earn, common_stock in zip(profit, commonStock)]
                
                try:
                    epsg = []
                    for ep in range(len(eps)):
                        if ep == 0:
                            continue
                        elif ep == 1:
                            epsg.append(round(100 * (eps[ep - 1] / eps[ep] - 1), 2))
                        elif ep == 2:
                            